<a href="https://colab.research.google.com/github/CUOTR/SQL-BANKING/blob/main/BANKING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREPARE DATA

In [33]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/bank.db')
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS addresses")
cursor.execute("""
CREATE TABLE addresses (
    AddressID INT PRIMARY KEY,
    Street VARCHAR(100),
    City VARCHAR(100),
    Country VARCHAR(100)
);
""")

cursor.execute("DROP TABLE IF EXISTS customer_types")
cursor.execute("""
CREATE TABLE customer_types (
    CustomerTypeID INT PRIMARY KEY,
    TypeName VARCHAR(50)
);
""")

cursor.execute("DROP TABLE IF EXISTS customers")
cursor.execute("""
CREATE TABLE customers (
    CustomerID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    DateOfBirth DATE,
    AddressID INT,
    CustomerTypeID INT,
    FOREIGN KEY (AddressID) REFERENCES addresses(AddressID),
    FOREIGN KEY (CustomerTypeID) REFERENCES customer_types(CustomerTypeID)
);
""")

cursor.execute("DROP TABLE IF EXISTS account_types")
cursor.execute("""
CREATE TABLE account_types (
    AccountTypeID INT PRIMARY KEY,
    TypeName VARCHAR(50)
);
""")

cursor.execute("DROP TABLE IF EXISTS account_statuses")
cursor.execute("""
CREATE TABLE account_statuses (
    AccountStatusID INT PRIMARY KEY,
    StatusName VARCHAR(50)
);
""")

cursor.execute("DROP TABLE IF EXISTS accounts")
cursor.execute("""
CREATE TABLE accounts (
    AccountID INT PRIMARY KEY,
    CustomerID INT,
    AccountTypeID INT,
    AccountStatusID INT,
    Balance DECIMAL(15,2),
    OpeningDate TEXT,
    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
    FOREIGN KEY (AccountTypeID) REFERENCES account_types(AccountTypeID),
    FOREIGN KEY (AccountStatusID) REFERENCES account_statuses(AccountStatusID)
);
""")

cursor.execute("DROP TABLE IF EXISTS transaction_types")
cursor.execute("""
CREATE TABLE transaction_types (
    TransactionTypeID INT PRIMARY KEY,
    TypeName VARCHAR(50)
);
""")

cursor.execute("DROP TABLE IF EXISTS branches")
cursor.execute("""
CREATE TABLE branches (
    BranchID INT PRIMARY KEY,
    BranchName VARCHAR(100),
    AddressID INT,
    FOREIGN KEY (AddressID) REFERENCES addresses(AddressID)
);
""")

cursor.execute("DROP TABLE IF EXISTS transactions")
cursor.execute("""
CREATE TABLE transactions (
    TransactionID INT PRIMARY KEY,
    AccountID INT,
    BranchID INT,
    TransactionTypeID INT,
    Amount DECIMAL(15,2),
    Description VARCHAR(255),
    TransactionDate DATETIME,
    AccountOriginID INT,
    AccountDestinationID INT,
    FOREIGN KEY (AccountID) REFERENCES accounts(AccountID),
    FOREIGN KEY (BranchID) REFERENCES branches(BranchID),
    FOREIGN KEY (TransactionTypeID) REFERENCES transaction_types(TransactionTypeID)
);
""")

cursor.execute("DROP TABLE IF EXISTS loan_statuses")
cursor.execute("""
CREATE TABLE loan_statuses (
    LoanStatusID INT PRIMARY KEY,
    StatusName VARCHAR(50)
);
""")

cursor.execute("DROP TABLE IF EXISTS loans")
cursor.execute("""
CREATE TABLE loans (
    LoanID INT PRIMARY KEY,
    AccountID INT,
    LoanStatusID INT,
    Amount DECIMAL(15,2),
    InterestRate DECIMAL(5,2),
    StartDate DATE,
    EndDate DATE,
    FOREIGN KEY (AccountID) REFERENCES accounts(AccountID),
    FOREIGN KEY (LoanStatusID) REFERENCES loan_statuses(LoanStatusID)
);
""")

conn.commit()

In [34]:
accounts = pd.read_csv('/content/accounts.csv')
account_types = pd.read_csv('/content/account_types.csv')
account_statuses = pd.read_csv('/content/account_statuses.csv')

customers = pd.read_csv('/content/customers.csv')
customer_types = pd.read_csv('/content/customer_types.csv')

loans = pd.read_csv('/content/loans.csv')
loan_statuses = pd.read_csv('/content/loan_statuses.csv')

transactions = pd.read_csv('/content/transactions.csv')
transaction_types = pd.read_csv('/content/transaction_types.csv')

branches = pd.read_csv('/content/branches.csv')
addresses = pd.read_csv('/content/addresses.csv')


cursor.execute("DROP TABLE IF EXISTS accounts")
cursor.execute("DROP TABLE IF EXISTS account_types")
cursor.execute("DROP TABLE IF EXISTS account_statuses")

cursor.execute("DROP TABLE IF EXISTS customers")
cursor.execute("DROP TABLE IF EXISTS customer_types")

cursor.execute("DROP TABLE IF EXISTS loans")
cursor.execute("DROP TABLE IF EXISTS loan_statuses")

cursor.execute("DROP TABLE IF EXISTS transaction_types")
cursor.execute("DROP TABLE IF EXISTS transactions")

cursor.execute("DROP TABLE IF EXISTS branches")
cursor.execute("DROP TABLE IF EXISTS addresses")

accounts.to_sql('accounts', conn, if_exists='append', index=False)
account_types.to_sql('account_types', conn, if_exists='append', index=False)
account_statuses.to_sql('account_statuses', conn, if_exists='append', index=False)

customers.to_sql('customers', conn, if_exists='append', index=False)
customer_types.to_sql('customer_types', conn, if_exists='append', index=False)

loans.to_sql('loans', conn, if_exists='append', index=False)
loan_statuses.to_sql('loan_statuses', conn, if_exists='append', index=False)

transactions.to_sql('transactions', conn, if_exists='append', index=False)
transaction_types.to_sql('transaction_types', conn, if_exists='append', index=False)

branches.to_sql('branches', conn, if_exists='append', index=False)
addresses.to_sql('addresses', conn, if_exists='append', index=False)

conn.commit()

EDA TRANSACTION

In [35]:
sql1 = '''
SELECT
    SUM(CASE WHEN TransactionID IS NULL THEN 1 ELSE 0 END) AS null_id,
    SUM(CASE WHEN Amount IS NULL THEN 1 ELSE 0 END) AS null_amount,
    SUM(CASE WHEN TransactionDate IS NULL THEN 1 ELSE 0 END) AS null_date,
    SUM(CASE WHEN TransactionTypeID IS NULL THEN 1 ELSE 0 END) AS null_type
FROM transactions;
'''
result = pd.read_sql_query(sql1, conn)
print(result)

   null_id  null_amount  null_date  null_type
0        0            0       1000          0


In [36]:
sql2 = '''
SELECT
    MIN(Amount) AS min_amount,
    MAX(Amount) AS max_amount,
    AVG(Amount) AS avg_amount,
    COUNT(*) AS total_transactions,
    MIN(TransactionDate) AS first_transaction_date,
    MAX(TransactionDate) AS last_transaction_date
FROM transactions;
'''
result = pd.read_sql_query(sql2, conn)
print(result)

   min_amount  max_amount   avg_amount  total_transactions  \
0        1.01     4999.59  2503.049865               49508   

  first_transaction_date last_transaction_date  
0          1/1/2020 0:00         9/9/2023 9:00  


In [37]:
sql3 = '''
SELECT
    t.TransactionID,
    t.Amount,
    t.TransactionDate,
    tt.TypeName,
    t.Description
FROM transactions t
JOIN transaction_types tt ON t.TransactionTypeID = tt.TransactionTypeID
WHERE tt.TypeName = 'Transfer'
ORDER BY t.Amount DESC
LIMIT 10;
'''
result = pd.read_sql_query(sql3, conn)
print(result)

   TransactionID   Amount   TransactionDate  TypeName        Description
0        3048536  4999.59   29/3/2021 22:00  Transfer  Transaction 48536
1        3048218  4999.40    30/5/2021 2:00  Transfer  Transaction 48218
2        3015481  4998.71   18/6/2020 14:00  Transfer  Transaction 15481
3        3001482  4998.29    16/2/2020 5:00  Transfer   Transaction 1482
4        3003065  4998.27    19/6/2022 2:00  Transfer   Transaction 3065
5        3009711  4997.70  17/10/2023 15:00  Transfer   Transaction 9711
6        3041474  4997.22   15/2/2022 23:00  Transfer  Transaction 41474
7        3013912  4997.16  27/10/2021 22:00  Transfer  Transaction 13912
8        3034759  4996.70     8/1/2024 5:00  Transfer  Transaction 34759
9        3023989  4996.67   17/7/2022 10:00  Transfer  Transaction 23989


In [38]:
sql3 = '''
SELECT
    tt.TypeName,
    MAX(t.TransactionDate) AS last_transaction_date
FROM transactions t
JOIN transaction_types tt ON t.TransactionTypeID = tt.TransactionTypeID
GROUP BY tt.TypeName
ORDER BY last_transaction_date DESC;
'''
result = pd.read_sql_query(sql3, conn)
print(result)

     TypeName last_transaction_date
0    Transfer         9/9/2023 9:00
1     Payment         9/9/2023 9:00
2     Deposit         9/9/2023 9:00
3  Withdrawal         9/9/2023 7:00


In [39]:
demo = '''
WITH monthly_tx AS (
    SELECT
        strftime('%Y-%m', TransactionDate) AS Month_Year,
        SUM(Amount) AS TotalAmount
    FROM transactions
    GROUP BY Month_Year
)
SELECT
    Month_Year,
    TotalAmount,
    ROUND((TotalAmount - LAG(TotalAmount, 1) OVER (ORDER BY Month_Year))
    / LAG(TotalAmount, 1) OVER (ORDER BY Month_Year) * 100, 2) AS MoM_GrowthPercent,
    ROUND((TotalAmount - LAG(TotalAmount, 12) OVER (ORDER BY Month_Year))
    / LAG(TotalAmount, 12) OVER (ORDER BY Month_Year) * 100, 2) AS YoY_GrowthPercent
FROM monthly_tx
ORDER BY Month_Year;
'''
result = pd.read_sql_query(demo, conn)
print(result)

  Month_Year   TotalAmount MoM_GrowthPercent YoY_GrowthPercent
0       None  1.239210e+08              None              None


In [40]:
query = '''
SELECT
    b.BranchName,
    COUNT(t.TransactionID) AS TransactionCount,
    SUM(t.Amount) AS TotalAmount
FROM transactions t
JOIN branches b ON t.BranchID = b.BranchID
GROUP BY b.BranchName
ORDER BY TotalAmount DESC, TransactionCount DESC
LIMIT 5;
 '''
result = pd.read_sql_query(query, conn)
print(result)

  BranchName  TransactionCount  TotalAmount
0  Branch 47              1046   2687234.07
1  Branch 39              1013   2630971.42
2  Branch 50              1064   2600262.96
3  Branch 46              1044   2599029.73
4  Branch 25              1029   2598472.03
